In [2]:
from qpms import Particle, CTMatrix, BaseSpec, FinitePointGroup, ScatteringSystem, TMatrixInterpolator, eV, hbar
from qpms.symmetries import point_group_info
import numpy as np
np.random.seed(666)

/home/mmn/.local/lib/python3.4/site-packages/quaternion/numba_wrapper.py:20: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [ ]:
sym = FinitePointGroup(point_group_info['D3h'])
bspec2 = BaseSpec(lMax=2)
bspec1 = BaseSpec(lMax=1)
t1 = CTMatrix(bspec1, np.diag(np.random.random(len(bspec1))))
t2 = CTMatrix(bspec2, np.diag(np.random.random(len(bspec2))))
p1 = Particle((1,2,),t1)
p2 = Particle((1,2,3),t1)
p3 = Particle((0.1,2),t2)
ss = ScatteringSystem([p1, p2, p3], sym)
ss.particles_tmi()

In [ ]:
tmfile = '/home/mmn/tmp/cylinder_50nm_lMax4_cleaned.TMatrix'
interp = TMatrixInterpolator(tmfile, bspec1)

In [1]:
interp

NameError: name 'interp' is not defined

In [ ]:
try:
    interp(3)
except ValueError as e:
    print("caught:", e)

In [ ]:
interp(3e14)[...]

In [ ]:
%time interp2 = TMatrixInterpolator(tmfile, bspec2, symmetrise = sym, atol = 1e-19)

In [8]:
interp2(3e14)[:6,:6]

array([[-5.75175232e-14+6.86394412e-05j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  8.35298188e-15+6.86097097e-05j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, -5.75175235e-14+6.86394412e-05j,  5.12981350e-14-6.25716090e-16j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  4.09698344e-12+7.10271500e-16j, -4.30117502e-17+2.82678081e-08j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, 

In [9]:
ss.fecv_size

204

In [10]:
ss.saecv_sizes, sum(ss.saecv_sizes)

([17, 17, 17, 68, 68, 17], 204)

In [11]:
ss.nirreps

6

In [12]:
ss.irrep_names

["A2''", "A2'", "A1'", "E'", "E''", "A1''"]

In [13]:
fullvector = np.random.rand(ss.fecv_size) + 1j*np.random.rand(ss.fecv_size)

In [14]:
packedvectors = [(iri, ss.pack_vector(fullvector, iri)) for iri in range(ss.nirreps)]

In [15]:
unpackedvectors = np.array([ss.unpack_vector(v[1], v[0]) for v in packedvectors])

In [16]:
rec_fullvector = np.sum(unpackedvectors, axis=0)

In [17]:
np.amax(abs(rec_fullvector-fullvector))

8.793919525157558e-15

In [18]:
packedmatrices = list()
for iri in range(ss.nirreps):
    d = ss.saecv_sizes[iri]
    m = np.random.rand(d,d)+1j*np.random.rand(d,d)
    packedmatrices.append((iri,m))

In [19]:
fullmatrix = np.zeros((ss.fecv_size, ss.fecv_size), dtype=complex)
for iri, m in packedmatrices:
    fullmatrix += ss.unpack_matrix(m, iri)

In [20]:
for iri, m in packedmatrices:
    print (m.shape)
    repackedmatrix = ss.pack_matrix(fullmatrix,iri)
    print(np.amax(abs(repackedmatrix-m)))
    

(17, 17)
1.6136471996107275e-15
(17, 17)
2.4650951625709315e-15
(17, 17)
1.6091054089879258e-15
(68, 68)
2.449144506106989e-15
(68, 68)
2.914203267236168e-15
(17, 17)
2.60133570547882e-15


In [21]:
iri = 1
d = ss.saecv_sizes[iri]
pm = np.random.rand(d,d)
fm = ss.unpack_matrix(pm,iri)
pmrec = ss.pack_matrix(fm,iri)
np.amax(abs(pm-pmrec))

7.105610857805792e-15

In [22]:
k = 1.7
modematrix_full = ss.modeproblem_matrix_full(k)
modematrix_packed_list = [(iri, ss.pack_matrix(modematrix_full,iri)) for iri in range(ss.nirreps)]
modematrix_full_rec = np.zeros((ss.fecv_size, ss.fecv_size), dtype=complex)
for iri, m in modematrix_packed_list:
    modematrix_full_rec += ss.unpack_matrix(m,iri)
print(np.amax(abs(modematrix_full-modematrix_full_rec)))

15300.102902277049


In [23]:
modematrix_full

array([[-1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, ...,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j, -1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, ...,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, -1.00000000e+00+0.00000000e+00j, ...,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [ 3.02914590e-17+6.48364251e-18j,  0.00000000e+00+0.00000000e+00j,  3.02914590e-17+6.48364251e-18j, ..., -1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 9.04763542e-02+1.93657340e-02j,  7.83485469e-18+1.67698746e-18j,  9.04763542e-02+1.93657340e-02j, .

In [24]:
k = 1.7
S_full = ss.translation_matrix_full(k)
S_packed_list = [(iri, ss.pack_matrix(S_full,iri)) for iri in range(ss.nirreps)]
S_full_rec = np.zeros((ss.fecv_size, ss.fecv_size), dtype=complex)
for iri, m in S_packed_list:
    S_full_rec += ss.unpack_matrix(m,iri)
print(np.amax(abs(S_full-S_full_rec)))

21036.24074892505


In [25]:
modematrix_full2 = ss.modeproblem_matrix_full(k)


In [26]:
np.amax(np.abs(modematrix_full-modematrix_full2))

0.0

In [ ]:
point_